In [ ]:
app = Flask(__name__)

session_id = 0

@app.route('/')
def index():
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">visprog</a></h1>
            <ol>
                <a href="/imgupload">이미지업로드</a>
            </ol>
        </body>
    </html>
    '''

@app.route('/imgupload')
def imgupload():
    global session_id 
    session_id = str(uuid.uuid4())
    return f'''<!doctype html>
    <html>
        <body>
            <form action="http://localhost:5000/command_image"
                method="POST"
                enctype="multipart/form-data">
                <input type="file" name="file" />
                <input type="submit" />
            </form>
        </body>
    </html>
    '''

@app.route('/command_image', methods=['POST'])
def imgUploader():
    s3 = uploads_utils.s3Connection()
    bucket = 'dear-image-flask'
    file = request.files['file']

    if file.filename == '':
        return 'No selected file'
    
    if uploads_utils.allowedFile(file.filename):
        filename = secure_filename(file.filename)

        s3_filepath = f'{session_id}/{filename}'
        s3.upload_fileobj(file, bucket, s3_filepath)

    # 이미지에 대하여 넘겨야 하는 command(명령) 관련 코드도 여기에 작성
    # 리다이렉션 어디로 해야할지..
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">File uploaded successfully</a></h1>
        </body>
    </html>
    '''


app.run()

